In [3]:
import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
import pandas as pd

def load_data(path='../../Files/', bow=False):
    # Load processed data
    filename = 'processed_data.csv' if not bow else 'processed_data_bow.csv'
    proc_df = pd.read_csv(path+filename)
    
    # Load predictions
    relevant_cols = ["PERMID", "CIK", "Ticker", "year", "FilingDate", "company_name", "Dividend Payer", "DPS growth", "DPS cut", "zEnvironmental", "dEnvironmental", "sector"]
    pred_df = pd.read_excel(path + "predictions.xlsx", sheet_name="data", skiprows=32, usecols=relevant_cols)
    pred_df.columns = ["perm_id", "cik", "ticker", "year", "filing_date", "company_name", "is_dividend_payer", "dps_change", "is_dps_cut", "z_environmental", "d_environmental", "sector"]
    pred_df['perm_id'] = pred_df['perm_id'].str.replace(r"[^0-9]",'')
    pred_df["filing_date"] = pd.to_datetime(pred_df["filing_date"])
    pred_df["filing_year"] = pd.DatetimeIndex(pred_df["filing_date"]).year
    
    return proc_df, pred_df

def preprocess_for_gensim(data):
    for section in ('item1a_risk', 'item7_mda'):
        doc_list = data[section]
        doc_tokenized = [simple_preprocess(doc) for doc in doc_list]
        dictionary = corpora.Dictionary()
        bow_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokenized]
        data[section+'_bow'] = bow_corpus
    
    return data